In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC


from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

import warnings

warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('train_case2.csv', ';')
df

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,99993,19240,2,168,76.0,120,80,1,1,1,0,1,0
69996,99995,22601,1,158,126.0,140,90,2,2,0,0,1,1
69997,99996,19066,2,183,105.0,180,90,3,1,0,1,0,1
69998,99998,22431,1,163,72.0,135,80,1,2,0,0,0,1


In [3]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio', 1), 
                                                    df['cardio'], random_state=0)

### Подготовка данных

In [4]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]


from sklearn.preprocessing import StandardScaler


continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

In [5]:
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[-1.73391771,  0.6873301 ,  0.74843904, ...,  1.        ,
         0.        ,  1.        ],
       [-1.67343538,  0.07758923, -0.29640123, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.13738132,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.17775864,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  1.        ],
       [-0.47190715, -1.38578883,  0.74843904, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.38174619,  0.56538192, -0.08743318, ...,  0.        ,
         0.        ,  1.        ]])

### Запустим обучение с кросс-валидацией для трех моделей

In [26]:
classifier1 = Pipeline([
('features',feats),
('classifier',  LogisticRegression(random_state = 42)),
])
 #запустим кросс-валидацию
cv_scores = cross_val_score(classifier1, X_train, y_train, cv=16, scoring='roc_auc')
cv_score = np.mean(cv_scores)
сv_score_std = np.std(cv_scores)
print('CV score is {}+-{}'.format(cv_score, cv_score_std))

CV score is 0.7867401104915408+-0.006337215867943306


In [27]:
classifier2 = Pipeline([
('features',feats),
('classifier', GradientBoostingClassifier(n_estimators=100, max_depth=3)),
 ])
 #запустим кросс-валидацию
cv_scores = cross_val_score(classifier2, X_train, y_train, cv=16, scoring='roc_auc')
cv_score = np.mean(cv_scores)
сv_score_std = np.std(cv_scores)
print('CV score is {}+-{}'.format(cv_score, cv_score_std))

CV score is 0.8025158426742907+-0.006337215867943306


In [28]:
classifier3 = Pipeline([
('features',feats),
('lassifier', KNeighborsClassifier(n_neighbors=3)),
])
 #запустим кросс-валидацию
cv_scores = cross_val_score(classifier3, X_train, y_train, cv=16, scoring='roc_auc')
cv_score = np.mean(cv_scores)
сv_score_std = np.std(cv_scores)
print('CV score is {}+-{}'.format(cv_score, cv_score_std))

CV score is 0.6715909806799311+-0.006337215867943306


Для всех моделей отклонение вышло приемлимым

### Напишем фукнцию для подсчета метрик, применим ее ко всем моделям и запишем результат в таблицу

In [33]:

def calc_metrics(df_metrics, model, name):
    
    model.fit(X_train, y_train)
    y_score = model.predict_proba(X_test)[:,1]
     
    b=1
    
    precision, recall, thresholds = precision_recall_curve(y_test, y_score)
    fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
    ix = np.argmax(fscore)
    roc_auc = roc_auc_score(y_true=y_test, y_score=model.predict_proba(X_test)[:,1])
    df_metrics.loc[len(df_metrics)] = [name, precision[ix], recall[ix], fscore[ix], roc_auc]
    return df_metrics

In [37]:
df_metrics = pd.DataFrame(columns = ['model', 'precision', 'recall', 'f_score', 'roc_auc'])
df_metrics = calc_metrics(df_metrics, classifier1, 'logreg')
df_metrics = calc_metrics(df_metrics, classifier2, 'gradboost')
df_metrics = calc_metrics(df_metrics, classifier3, 'randforest')
df_metrics

,model,precision,recall,f_score,roc_auc
0,logreg,0.647431,0.837558,0.730323,0.784035
1,gradboost,0.697848,0.788134,0.740248,0.802615
2,randforest,0.557032,0.874309,0.680506,0.670882


из таблицы видно, что наилучший результат показал градиентный бустинг.
У случайного леса и логистической регресси видна низкая полнота, и достаточно большая точность. Значит достаточно большое число пациентов с сердечно-сосудистым заболеванием не получат раннюю диагнстику.